<a href="https://colab.research.google.com/github/chaitanya-7741/AI-agents/blob/main/calculator3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install SpeechRecognition gTTS num2words pyaudio
!pip install playsound

  Preparing metadata (setup.py) ... done
  Created wheel for playsound: filename=playsound-1.3.0-py3-none-any.whl size=7020 sha256=3a215482b5451be0e911e38248d3b360c0919c20378a27d91b57dc4bda9158f1
  Stored in directory: /root/.cache/pip/wheels/50/98/42/62753a9e1fb97579a0ce2f84f7db4c21c09d03bb2091e6cef4
Successfully built playsound


In [8]:
!apt-get update
!apt-get install -y python3-dev portaudio19-dev
!pip install pyaudio

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [13]:
import speech_recognition as sr
from gtts import gTTS
from num2words import num2words
import re
import os
import playsound

# Mapping of spoken words to operators
replacements = {
    "plus": "+",
    "minus": "-",
    "times": "*",
    "multiplied by": "*",
    "x": "*",
    "divide by": "/",
    "divided by": "/",
    "over": "/",
    "to the power of": "**"
}

def parse_spoken(text):
    text = text.lower()
    for word, symbol in replacements.items():
        text = text.replace(word, symbol)
    return text

def expression_to_words(expr):
    expr = expr.replace('**', ' ** ')
    tokens = re.findall(r'\d+\.?\d*|[()+\-*/%]+', expr)

    symbol_map = {
        '+': 'plus',
        '-': 'minus',
        '*': 'times',
        '/': 'divided by',
        '**': 'to the power of',
        '//': 'floor divided by',
        '%': 'modulus',
        '(': 'open parenthesis',
        ')': 'close parenthesis'
    }

    words = []
    for token in tokens:
        if token in symbol_map:
            words.append(symbol_map[token])
        elif token.replace('.', '', 1).isdigit():
            words.append(num2words(float(token)))
        else:
            words.append(token)

    return ' '.join(words)

def speak(text, filename="response.mp3"):
    tts = gTTS(text)
    tts.save(filename)
    # In a server environment without audio output, playing sound might not work.
    # You might need to download the file or handle playback differently.
    # For now, we'll keep the playsound call, but be aware it might fail.
    try:
        playsound.playsound(filename)
    except Exception as e:
        print(f"Warning: Could not play sound. {e}")
    os.remove(filename)

# --- Changes start here ---

# Step 1: Get audio input from a file
r = sr.Recognizer()

# Define the path to your audio file
# Make sure you upload an audio file (e.g., a .wav file) to your environment
# and update the file_path variable accordingly.
# Example: If you upload 'math_expression.wav' to your current directory
file_path = "math_expression.wav" # <--- Change this to your audio file name

try:
    with sr.AudioFile(file_path) as source:
        print(f"🎧 Reading audio from {file_path}...")
        audio = r.listen(source)

    # Step 2: Recognize speech
    # Use a recognizer (like Google Web Speech API) to convert audio to text
    spoken_text = r.recognize_google(audio)
    print("Recognized text:", spoken_text)

    # Step 3: Parse and evaluate
    parsed_expr = parse_spoken(spoken_text)
    # Added a basic check to prevent eval from executing potentially harmful code
    # This is a minimal safeguard; for a production application, use a safer parser
    if any(char.isalpha() and char not in replacements.keys() for char in parsed_expr):
         raise ValueError("Invalid characters detected in expression.")
    result = eval(parsed_expr)


    # Step 4: Convert to words and speak (generates an audio file)
    expr_words = expression_to_words(parsed_expr)
    result_words = num2words(result)

    full_sentence = f"{expr_words} equals {result_words}"
    print(f"🧮 Result: {result}")
    print("🔊 Generating audio:", full_sentence)
    speak(full_sentence) # This saves and tries to play an MP3 file

except sr.UnknownValueError:
    print("Speech Recognition could not understand audio")
    speak("Sorry, I did not understand that.")

except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")
    speak("Sorry, there was an issue contacting the speech recognition service.")

except FileNotFoundError:
    print(f"Error: Audio file not found at {file_path}")
    speak("Sorry, the audio file was not found.")

except Exception as e:
    print("Error:", e)
    speak("Sorry, there was an error processing the expression.")

# --- Changes end here ---

Error: Audio file not found at math_expression.wav
